1. Install cmake: `conda install -c anaconda cmake -y`
2. Install setup: `python setup.py install`


At the end

```python
# Install linters
pip install pydocstyle pycodestyle pytest

# Check tests
pytest -v tests/python_package_test/test_engine.py

# Check linters
pycodestyle --ignore=E501,W503 python-package/lightgbm/basic.py
pycodestyle --ignore=E501,W503 tests/python_package_test/test_engine.py
pydocstyle --convention=numpy --add-ignore=D105 python-package/lightgbm/basic.py
```

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import log_loss

import lightgbm as lgb
import numpy as np

In [15]:
from lightgbm import __version__

print(__version__)

3.3.1.99


In [12]:
X, y = load_breast_cancer(return_X_y=True)
lgb_train = lgb.Dataset(X, y, init_score=np.zeros(y.size))
train_params = {
    'objective': 'binary',
    'verbose': -1,
    'seed': 123
}
gbm = lgb.train(train_params, lgb_train, num_boost_round=10)
non_weight_err_pred = log_loss(y, gbm.predict(X))
refit_weight = np.random.rand(y.shape[0])
dataset_params = {
    'max_bin': 260,
    'min_data_in_bin': 5,
    'data_random_seed': 123,
}
new_gbm = gbm.refit(
    data=X,
    label=y,
    weight=refit_weight,
    dataset_params=dataset_params,
    decay_rate=0.0,
)
weight_err_pred = log_loss(y, new_gbm.predict(X))
train_set_params = new_gbm.train_set.get_params()
stored_weights = new_gbm.train_set.get_weight()
assert weight_err_pred != non_weight_err_pred
assert train_set_params["max_bin"] == 260
assert train_set_params["min_data_in_bin"] == 5
assert train_set_params["data_random_seed"] == 123
np.testing.assert_allclose(stored_weights, refit_weight)

In [13]:
weight_err_pred

0.25130293183342284

In [14]:
non_weight_err_pred

0.2520013338599355